In [60]:
import pandas as pd
import altair as alt
import warnings
import requests
import datetime
import matplotlib.pyplot as plt
import json
import requests
alt.renderers.set_embed_options(theme='dark')
pd.set_option('display.max_colwidth', None)

In [61]:
class AstroDataProvider:
    
    def __init__(self, claim):
        daic_url = "https://terra-api.daic.capital/api/tx/GetRichlistByTokenContract?apiKey=vAp6ysmAXH470YcphYxv&contract_address={}"
        self.votes = '4940a215-6e93-4107-bf08-50574b3e431d'
        self.astro_holders_url =daic_url.format("terra1xj49zyqrwpv5k928jwfpfy2ha668nwdgkwlrg3")
        self.xastro_holders_url =daic_url.format("terra14lpnyzc9z4g3ugr4lhm8s4nle0tq8vcltkhzh7")
        self.claim = claim
        
    def get_from_url(self, url):
        json = requests.get(url).json()
        return json
        
    def load(self):
        self.votes_df = self.claim(self.votes)
        #
        json = self.get_from_url(self.astro_holders_url)['result']['holders']
        self.astro_holders_df = pd.DataFrame(json.values(),json.keys()).reset_index()
        self.astro_holders_df.columns =  ['addr','amount']
        #
        json = self.get_from_url(self.xastro_holders_url)['result']['holders']
        self.xastro_holders_df = pd.DataFrame(json.values(),json.keys()).reset_index()
        self.xastro_holders_df.columns =  ['addr','amount']
        #
        
        
    def parse(self):
        self.astro_holders_df.amount = self.astro_holders_df.amount/1000000
        
    def to_csv(self, path='../data'):
        self.votes_df.to_csv(f"{path}/votes_df.csv")
        self.astro_holders_df.to_csv(f"{path}/astro_holders_df.csv")
        
    def read_csv(self):
        url = 'https://raw.githubusercontent.com/IncioMan/nebula_lbp/master/data/{}.csv'
        self.ust_traded_prices_df =  pd.read_csv(url.format('ust_traded_prices_df'), index_col=0)


In [62]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [63]:
dp = AstroDataProvider(claim)
dp.load()
dp.parse()

In [64]:
dp.astro_holders_df[dp.astro_holders_df.amount>1000000].addr.values

array(['terra1c7m6j8ya58a2fkkptn8fgudx8sqjqvc8azq0ex',
       'terra1fh27l8h4s0tfx9ykqxq5efq4xx88f06x6clwmr',
       'terra1f68wt2ch3cx2g62dxtc8v68mkdh5wchdgdjwz7',
       'terra1hncazf652xa0gpcwupxfj6k4kl4k4qg64yzjyf',
       'terra1l7xu2rl3c7qmtx3r5sd2tz25glf6jh8ul7aag7',
       'terra1627ldjvxatt54ydd3ns6xaxtd68a2vtyu7kakj',
       'terra1dpe2aqykm2vnakcz4vgpha0agxnlkjvgfahhk7',
       'terra12u7hcmpltazmmnq0fvyl225usn3fy6qqlp05w0',
       'terra17k5vg2zhuusxw9ngjs3482neze43k6cs60a7wv',
       'terra1p54hc4yy2ajg67j645dn73w3378j6k05v52cnk',
       'terra1590axydgjqv7wzt0l3zd2c8l4947pq42f8vatj'], dtype=object)

In [65]:
dp.votes_df[dp.votes_df.voter.isin(dp.astro_holders_df[dp.astro_holders_df.amount>10000].addr.values)]

,block_timestamp,tx_id,action,proposal_id,vote,voter,voting_power
3402,2022-04-21 08:26:48.747,E4ADBBDF7AE325F273261420D9C114E3B21A0F6C2A2BCAECFD49978A125DFE53,cast_vote,4,for,terra138jz60f7z59c2u90vxv0gf862trrqjqg25njjr,3.715647e+10


In [66]:
dp.votes_df[(dp.votes_df.voting_power>3012300000000)&(dp.votes_df.vote=='against')]

,block_timestamp,tx_id,action,proposal_id,vote,voter,voting_power
3085,2022-04-20 18:17:13.813,511D85CFB1DF4F0F11407A24A1E52F228345D1D2A9D976BE735F33B8A585DD4D,cast_vote,4,against,terra1jyxljeq865wsndutyyr0vrzzq9tj3fa6rmhgrl,3.774557e+12
4738,2022-04-21 20:22:20.004,7D97996C1A387A58028B655BC7345D9053CA09EE815751EFFDFABE4DD52B55E8,cast_vote,3,against,terra1jyxljeq865wsndutyyr0vrzzq9tj3fa6rmhgrl,3.774557e+12
4744,2022-04-21 20:12:36.912,32BEC3378FFA792C68D53DBA8B3582AB4226C6FC7FD429BB617500A23769B0B1,cast_vote,3,against,terra1qn93rzsnfh0dwdkxmf8q6dtjr0uxv33klhlc5u,3.403807e+12
5339,2022-04-23 16:05:03.655,8BFBD276B3464A8FEA6CA47E75DD7C114BC1C6DD8132EDDB84D5663ED6E56B8B,cast_vote,5,against,terra1qn93rzsnfh0dwdkxmf8q6dtjr0uxv33klhlc5u,3.403807e+12
5395,2022-04-22 14:46:11.877,DB0EB3EDD43773AC4AE11EC5046947065E347C9E1F95A3D0060418810EB02E0A,cast_vote,3,against,terra1zaqeperrwghqlsa9yykzsjaets54mtq0u6kl60,1.500000e+13


In [70]:
dp.xastro_holders_df[dp.xastro_holders_df.addr=='terra1qn93rzsnfh0dwdkxmf8q6dtjr0uxv33klhlc5u']

,addr,amount
2,terra1qn93rzsnfh0dwdkxmf8q6dtjr0uxv33klhlc5u,3403806671336


In [395]:
class NebulaChartProvider:
    
    def ust_traded_prices_chart(self, ust_traded_prices):
        chart = alt.Chart(ust_traded_prices).mark_point().encode(
        x=alt.X('Price:Q', sort=alt.EncodingSortField(order='ascending')),
        y="Amount UST (M):Q",
        color=alt.Color('Action:N', scale=alt.Scale(domain=['Sold NEB','Bought NEB'],
                                                      range=['#F24A72','#21bcd7'])),
        tooltip=['Action','Amount UST (M):N','Price:Q']
        ).configure_mark(
            color='#21bcd7'
        ).properties(width=700).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def first_price_chart(self,df):
        cols = ['Number of Users','Price']
        chart = alt.Chart(df).mark_line(point=True).encode(
            y=alt.Y(cols[0]+":Q"),
            x=alt.X(cols[1]+":Q",axis=alt.Axis(tickCount=20, labelAngle=0, tickBand = 'center')),
            tooltip=[cols[0],cols[1]]
        ).configure_mark(
            color='#21bcd7'
        ).properties(height=300).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def first_time_chart(self,df):
        cols = ['Number of Users','Time'] 
        chart = alt.Chart(df).mark_bar().encode(
            y=alt.Y(cols[0]+":Q"),
            x=alt.X(cols[1]+":T"),
            tooltip=[alt.Tooltip(cols[1]+':T', format='%Y-%m-%d %H:%M'), alt.Tooltip(cols[0]+":Q")]
        ).configure_mark(
            color='#21bcd7'
        ).properties(height=300).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def n_prices_per_users_df_chart(self,df):
        cols = ['Number of Users','Number of Different Prices']
        chart = alt.Chart(df).mark_bar().encode(
            y=alt.Y(cols[0]+":Q"),
            x=alt.X(cols[1]+":N",axis=alt.Axis(tickCount=10, labelAngle=30, tickBand = 'center')),
            tooltip=[cols[1], cols[0]]
        ).configure_mark(
            color='#21bcd7'
        ).properties(height=300).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def user_distr_pie(self, df, cols):
        chart = alt.Chart(df).mark_arc(innerRadius=60).encode(
            theta=alt.Theta(field=cols[0], type="quantitative"),
            color=alt.Color(field=cols[1], type="nominal",
                    #sort=['MARS & UST','MARS','UST'],
                    scale=alt.Scale(domain=df[cols[1]].unique(), range=['#F24A72','#21bcd7']),
                    legend=alt.Legend(
                    orient='none',
                    padding=10,
                    legendY=-10,
                    direction='vertical')),
            tooltip=[cols[1]+':N',cols[0]+':N']
        ).configure_view(strokeOpacity=0)
        return chart
    
    def sender_airdrop_op_charts(self, df, cols):
        df.columns = cols
        chart = alt.Chart(df).mark_arc(innerRadius=60).encode(
                    theta=alt.Theta(field=cols[1], type="quantitative"),
                    color=alt.Color(field=cols[0], type="nominal",
                            #sort=['MARS & UST','MARS','UST'],
                            scale=alt.Scale(domain=df[cols[0]].unique(), range=['#ffffff','#21bcd7','#F24A72']),
                            legend=alt.Legend(
                            orient='none',
                            padding=10,
                            legendY=-10,
                            direction='vertical')),
                    tooltip=[cols[1]+':N',cols[0]+':N']
                ).configure_view(strokeOpacity=0)
        return chart
    
    def price_chart(self,hourly_stats_df):
        #272231 background
        df=hourly_stats_df[['avg_belief_price','time']]
        df.columns=['Price','Hour']
        n_data = 20
        if df.Hour.nunique() < n_data:
            extra_data = []
            for i in range(n_data-df.Hour.nunique()):
                extra_data.append([None,(pd.to_datetime(df.Hour.max())+datetime.timedelta(hours=i)).strftime("%Y-%m-%d %H:%M")])
            df2 = df.append(pd.DataFrame(extra_data, columns=df.columns))
        else:
            df2 = df
        chart = alt.Chart(df2).mark_line(point=True).encode(
            x=alt.X('Hour:T', sort=alt.EncodingSortField(order='ascending')),
            y="Price:Q",
            tooltip=['Hour:T',"Price:Q"]
        ).configure_mark(
            color='#21bcd7'
        ).properties(width=700).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart

dp = NebulaLBPProvider(claim)
dp.load()
dp.parse()

In [396]:
dp = NebulaLBPProvider(claim)
dp.read_csv()

In [397]:
df = dp.hourly_stats_df
maxh = df[df.avg_belief_price.notna()].time.apply(lambda x: str(x)).max()
currprice = df[df.time==maxh].avg_belief_price.values[0]
currprice

0.7000000000000001

In [398]:
cp = NebulaChartProvider()
cp.price_chart(dp.hourly_stats_df)

/var/folders/pd/3vx1d3qj29z1l1tm70p573s80000gn/T/ipykernel_8432/1297805603.py:94: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df.append(pd.DataFrame(extra_data, columns=df.columns))


alt.Chart(...)

In [399]:
cp.sender_airdrop_op_charts(dp.amount_airdropped_dumped_df, dp.amount_airdropped_dumped_df.columns)

alt.Chart(...)

In [400]:
cp.sender_airdrop_op_charts(dp.sender_airdrop_op_df, dp.sender_airdrop_op_df.columns)

alt.Chart(...)

In [401]:
cp.user_distr_pie(dp.airdrop_in_lbp, dp.airdrop_in_lbp.columns)

alt.Chart(...)

In [402]:
cp.user_distr_pie(dp.lbp_from_airdrop, dp.lbp_from_airdrop.columns)

alt.Chart(...)

In [403]:
cp.n_prices_per_users_df_chart(dp.n_prices_per_users_df)

alt.Chart(...)

In [404]:
cp.ust_traded_prices_chart(dp.ust_traded_prices_df)

alt.Chart(...)

In [405]:
cp.first_time_chart(dp.first_time_parse_df)

alt.Chart(...)

In [406]:
cp.first_price_chart(dp.first_price_parse_df)

alt.Chart(...)